#Preprocessing

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

[](http://)Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [160]:
# read data from file
train = pd.read_csv("/content/train.csv") 
test = pd.read_csv("/content/test.csv")


# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [161]:
train.head()

,PhoneId,Also Known As,Applications,Architecture,Aspect Ratio,Audio Features,Audio Jack,Autofocus,Bezel-less display,Bluetooth,Brand,Browser,Build Material,Camera Features,Capacity,Chipset,Co-Processor,Colours,Custom UI,Display Colour,Display Type,Email,Expandable Memory,FM Radio,Fingerprint Sensor,Fingerprint Sensor Position,Flash,GPS,Games,Graphics,HDMI,Heart Rate Monitor,Height,IRIS Scanner,Image Resolution,Internal Memory,Java,Launch Date,Loudspeaker,Mobile High-Definition Link(MHL),...,Ruggedness,SAR Value,SIM 1,SIM 2,SIM 3,SIM Size,SIM Slot(s),SMS,Screen Protection,Screen Resolution,Screen Size,Screen to Body Ratio (calculated),Screen to Body Ratio (claimed by the brand),Sensor,Settings,Shooting Modes,Software Based Aperture,Special Features,Standby time,Stylus,TalkTime,Thickness,Touch Screen,Type,USB Connectivity,USB OTG Support,USB Type-C,User Available Storage,User Replaceable,Video Player,Video Recording,Video Recording Features,VoLTE,Waterproof,Weight,Wi-Fi,Wi-Fi Features,Width,Wireless Charging,Rating
0,0,NaN,NaN,64 bit,19:9,NaN,3.5 mm,Phase Detection autofocus,yes,v5.0,Xiaomi,NaN,NaN,"Digital Zoom, Auto Flash, Face detection, Touc...",4000 mAh,Qualcomm Snapdragon 636,NaN,"Black, Blue, Red, Rose Gold",MIUI,NaN,IPS LCD,NaN,Up to 256 GB,NaN,yes,Rear,Dual-color LED Flash,"with A-GPS, Glonass",NaN,Adreno 509,NaN,NaN,157.9 mm Compare Size,NaN,4000 x 3000 Pixels,64 GB Best in Class ▾Internal Memory compared ...,NaN,"November 23, 2018 (Official)",yes,NaN,...,NaN,NaN,4G Bands:TD-LTE 2300(band 40) / 2500(band 41) ...,4G Bands: TD-LTE 2300(band 40) / 2500(band 41)...,NaN,"SIM1: Nano, SIM2: Nano (Hybrid)","Dual SIM, GSM+GSM, Dual VoLTE",NaN,Corning Gorilla Glass,1080 x 2280 pixels,6.26 inches (15.9 cm),80.68 %,NaN,Exmor RS,"Exposure compensation, ISO control","Continuos Shooting, High Dynamic Range mode (H...",NaN,NaN,NaN,NaN,NaN,8.2 mm Very Good ▾Thickness compared to11 - 17...,"Capacitive Touchscreen, Multi-touch",Li-Polymer,"Mass storage device, USB charging, microUSB 2.0",yes,NaN,Up to 52.3 GB,No,NaN,1920x1080 @ 30 fps,NaN,yes,NaN,182 grams Below Average ▾Weight compared to11 ...,"Wi-Fi 802.11, a/b/g/n","Wi-Fi Direct, Mobile Hotspot",76.4 mm,NaN,4.5
1,1,NaN,Oppo Browser,64 bit,19:9,NaN,3.5 mm,Phase Detection autofocus,yes,v4.2,Realme,NaN,NaN,"2 x Digital Zoom, Auto Flash, Face detection, ...",4230 mAh,Qualcomm Snapdragon 450,NaN,"Diamond Black, Diamond Blue, Diamond Red",ColorOS,NaN,IPS LCD,NaN,Up to 256 GB,NaN,yes,Rear,LED Flash,with A-GPS,NaN,Adreno 506,NaN,NaN,156.2 mm Compare Size,NaN,4128 x 3096 Pixels,32 GB Excellent ▾Internal Memory compared to8 ...,NaN,"September 4, 2018 (Official)",yes,NaN,...,NaN,NaN,4G Bands:TD-LTE 2600(band 38) / 2300(band 40) ...,4G Bands: TD-LTE 2600(band 38) / 2300(band 40)...,NaN,"SIM1: Nano, SIM2: Nano","Dual SIM, GSM+GSM, Dual VoLTE",NaN,Corning Gorilla Glass v3,720 x 1520 pixels,6.2 inches (15.75 cm),80.85 %,88.8 %,NaN,"Exposure compensation, ISO control","Continuos Shooting, High Dynamic Range mode (HDR)",NaN,NaN,NaN,NaN,Up to 44 Hours(2G),8.2 mm Very Good ▾Thickness compared to8 - 12 ...,"Capacitive Touchscreen, Multi-touch",Li-ion,"Mass storage device, USB charging, microUSB 2.0",yes,NaN,Up to 23.2 GB,No,NaN,1920x1080 @ 30 fps,NaN,yes,NaN,168 grams Average ▾Weight compared to8 - 12 K ...,"Wi-Fi 802.11, b/g/n",Mobile Hotspot,75.6 mm,NaN,4.5
2,2,NaN,NaN,64 bit,19.5:9,NaN,3.5 mm,yes,yes,v4.2,Realme,NaN,NaN,"2 x Digital Zoom, Auto Flash, Face detection, ...",3500 mAh,MediaTek Helio P70,NaN,"Brave Blue, Fiery Gold, Ambitious Black",ColorOS,NaN,IPS LCD,NaN,Up to 256 GB,yes,yes,Rear,LED Flash,"with A-GPS, Glonass",NaN,Mali-G72 MP3,NaN,NaN,157 mm Compare Size,NaN,4128 x 3096 Pixels,32 GB Excellent ▾Internal Memory compared to10...,NaN,"December 5, 2018 (Official)",yes,NaN,...,NaN,NaN,4G Bands:TD-LTE 2600(band 38) / 2300(band 40) ...,4G Bands: TD-LTE 2600(band 38) / 2300(band 40)...,NaN,"SIM1: Nano, SIM2: Nano","Dual SIM, GSM+GSM, Dual VoLTE",NaN,Corning Gorilla Glass v3,1080 x 2340 pixels,6.

In [162]:
train.columns

Index(['PhoneId', 'Also Known As', 'Applications', 'Architecture',
       'Aspect Ratio', 'Audio Features', 'Audio Jack', 'Autofocus',
       'Bezel-less display', 'Bluetooth', 'Brand', 'Browser', 'Build Material',
       'Camera Features', 'Capacity', 'Chipset', 'Co-Processor', 'Colours',
       'Custom UI', 'Display Colour', 'Display Type', 'Email',
       'Expandable Memory', 'FM Radio', 'Fingerprint Sensor',
       'Fingerprint Sensor Position', 'Flash', 'GPS', 'Games', 'Graphics',
       'HDMI', 'Heart Rate Monitor', 'Height', 'IRIS Scanner',
       'Image Resolution', 'Internal Memory', 'Java', 'Launch Date',
       'Loudspeaker', 'Mobile High-Definition Link(MHL)', 'Model', 'Music',
       'NFC', 'Network', 'Network Support', 'Operating System',
       'Optical Image Stabilisation', 'Other Facilities', 'Other Sensors',
       'Phone Book', 'Physical Aperture', 'Pixel Density', 'Processor',
       'Quick Charging', 'RAM', 'Rating Count', 'Resolution', 'Review Count',
       'Ring

In [163]:
#Returns a list with the number of null values in the dataset, feature wise
list(train.isnull().sum(axis = 0))

[0,
 338,
 312,
 82,
 151,
 325,
 12,
 22,
 201,
 4,
 1,
 332,
 253,
 7,
 1,
 26,
 334,
 1,
 185,
 340,
 1,
 347,
 2,
 74,
 20,
 136,
 13,
 22,
 331,
 30,
 338,
 350,
 15,
 351,
 7,
 18,
 353,
 24,
 1,
 354,
 1,
 332,
 97,
 1,
 1,
 21,
 165,
 329,
 24,
 329,
 69,
 1,
 19,
 94,
 15,
 0,
 7,
 0,
 329,
 318,
 233,
 1,
 32,
 353,
 20,
 1,
 352,
 169,
 1,
 1,
 15,
 321,
 177,
 56,
 43,
 354,
 343,
 246,
 354,
 191,
 15,
 21,
 2,
 18,
 120,
 278,
 206,
 18,
 341,
 87,
 341,
 82,
 295,
 30,
 21,
 26,
 15,
 330,
 0]

In [164]:
train['Also Known As'].value_counts()

Honor View 10                 1
HTC Bolt                      1
Vivo V11 Pro                  1
Oppo RX17                     1
HTC Desire 826 Dual SIM       1
Vivo X20                      1
Oppo RealMe 1                 1
Samsung SM-G7102              1
Vivo X20 Plus                 1
Mobiistar XQ Dual             1
Xiaomi Mi 6X                  1
Nokia X5                      1
Blackberry KEY2 LE            1
Samsung Galaxy A9s            1
Nokia X7                      1
Vivo V9 Pro                   1
Samsung Guru Music 2 B310E    1
Name: Also Known As, dtype: int64

In [165]:
#Only describale features in the dataset for now are the 4 marked here.

train.describe()

,PhoneId,Rating Count,Review Count,Rating
count,355.000000,3.550000e+02,355.000000,355.000000
mean,235.267606,3.344979e+04,6648.228169,4.071549
std,137.204594,1.151661e+05,18943.317868,0.372663
min,0.000000,1.000000e+00,1.000000,2.500000
25%,113.500000,6.585000e+02,228.000000,3.900000
50%,234.000000,3.744000e+03,951.000000,4.100000
75%,358.000000,1.613650e+04,4208.500000,4.300000
max,472.000000,1.268083e+06,218020.000000,5.000000


In [166]:
train['Rating']

0      4.5
1      4.5
2      4.4
3      4.3
4      4.4
5      4.4
6      4.5
7      4.3
8      4.1
9      4.3
10     4.5
11     4.5
12     4.0
13     4.4
14     4.4
15     4.4
16     4.4
17     4.5
18     4.4
19     4.4
20     4.4
21     4.4
22     4.5
23     4.4
24     4.3
25     4.2
26     4.3
27     4.3
28     4.6
29     4.4
      ... 
325    3.9
326    4.4
327    4.3
328    4.1
329    4.1
330    3.9
331    3.4
332    3.3
333    3.9
334    4.0
335    4.4
336    4.5
337    4.2
338    3.8
339    3.7
340    4.2
341    4.1
342    4.0
343    3.9
344    3.7
345    4.2
346    4.0
347    3.6
348    3.6
349    4.4
350    3.8
351    3.9
352    4.1
353    3.6
354    4.3
Name: Rating, Length: 355, dtype: float64

In [167]:
train['Browser'].isnull().sum()

332

In [0]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

In [0]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [0]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [171]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 46
Number of data points in test: 119
Number of features in test: 45


In [172]:
train['Browser'].isnull().sum()

328

In [0]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [0]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [175]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 50
Number of data points in test: 119
Number of features in test: 49


Not very important feature

In [0]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [177]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [178]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'Num_cores', 'Resolution', 'Internal Memory', 'Screen Size',
       'SIM 2', 'Brand', 'Processor_frequency', 'SIM Slot(s)', 'Pixel Density',
       'Height', 'RAM', 'Capacity', 'Screen to Body Ratio (calculated)',
       'Weight', 'os_name', 'Sim1'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Resolution', 'Internal Memory', 'Screen Size',
       'Processor_frequency', 'Pixel Density', 'Height', 'RAM', 'Capacity',
       'Screen to Body Ratio (calculated)', 'Weight', 'Num_cores_312',
       'Num_cores_Deca', 'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa',
       'Num_cores_Other', 'Num_cores_Quad', 'Num_cores_Tru-Octa', 'SIM 2_2G',
       'SIM 2_3G', 'SIM 2_4G', 'SIM 2_Other', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Google',
       'Brand_HTC', 'Brand_Honor', 'Brand_Huawei', 'Brand_InFocus',
       'Brand_Infinix', 'Brand_Int

In [0]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [180]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [181]:
train_new.columns

Index(['PhoneId', 'Resolution', 'Internal Memory', 'Screen Size',
       'Processor_frequency', 'Pixel Density', 'Height', 'RAM', 'Capacity',
       'Screen to Body Ratio (calculated)', 'Weight', 'Num_cores_312',
       'Num_cores_Deca', 'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa',
       'Num_cores_Other', 'Num_cores_Quad', 'Num_cores_Tru-Octa', 'SIM 2_2G',
       'SIM 2_3G', 'SIM 2_4G', 'SIM 2_Other', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Google',
       'Brand_HTC', 'Brand_Honor', 'Brand_Huawei', 'Brand_InFocus',
       'Brand_Infinix', 'Brand_Intex', 'Brand_Itel', 'Brand_Jivi',
       'Brand_Karbonn', 'Brand_LG', 'Brand_Lava', 'Brand_LeEco',
       'Brand_Lenovo', 'Brand_Lephone', 'Brand_Lyf', 'Brand_Meizu',
       'Brand_Micromax', 'Brand_Mobiistar', 'Brand_Moto', 'Brand_Motorola',
       'Brand_Nokia', 'Brand_Nubia', 'Brand_OPPO', 'Brand_OnePlus',
 

In [182]:
test_new.columns

Index(['PhoneId', 'Resolution', 'Internal Memory', 'Screen Size',
       'Processor_frequency', 'Pixel Density', 'Height', 'RAM', 'Capacity',
       'Screen to Body Ratio (calculated)', 'Weight', 'Num_cores_312',
       'Num_cores_Deca', 'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa',
       'Num_cores_Other', 'Num_cores_Quad', 'Num_cores_Tru-Octa', 'SIM 2_2G',
       'SIM 2_3G', 'SIM 2_4G', 'SIM 2_Other', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Google',
       'Brand_HTC', 'Brand_Honor', 'Brand_Huawei', 'Brand_InFocus',
       'Brand_Infinix', 'Brand_Intex', 'Brand_Itel', 'Brand_Jivi',
       'Brand_Karbonn', 'Brand_LG', 'Brand_Lava', 'Brand_LeEco',
       'Brand_Lenovo', 'Brand_Lephone', 'Brand_Lyf', 'Brand_Meizu',
       'Brand_Micromax', 'Brand_Mobiistar', 'Brand_Moto', 'Brand_Motorola',
       'Brand_Nokia', 'Brand_Nubia', 'Brand_OPPO', 'Brand_OnePlus',
 

In [183]:
train_new.head()

,PhoneId,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,...,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,Rating
0,0,20,64,6.26,1.8,403,157.9,4,4000,80.68,182,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,4.5
1,1,8,32,6.20,1.8,271,156.2,3,4230,80.85,168,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,4.5
2,2,25,32,6.30,2.1,409,157.0,3,3500,83.68,168,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,4.4
3,4,24,64,6.00,2.2,411,159.8,4,3300,74.78,169,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,4.3
4,5,16,64,6.50,2.2,396,160.4,4,3750,84.23,175,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,4.4


In [184]:
test_new.head()

,PhoneId,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,...,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G
0,3,5,16,6.20,1.8,271,156.2,2,4230,80.85,168,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
1,11,12,64,6.20,1.8,402,156.0,4,5000,81.60,205,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
2,13,25,64,6.30,2.0,409,156.7,6,3500,83.84,169,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
3,16,16,64,6.30,2.0,409,156.7,4,3500,83.84,169,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
4,19,20,64,5.99,1.8,403,158.6,4,4000,77.43,181,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1


In [185]:
train_new.describe()

,PhoneId,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,...,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,Rating
count,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.0,341.000000,341.000000,341.000000,341.0,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,...,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.0,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000
mean,232.970674,9.592375,46.369501,5.462317,1.792551,344.211144,149.946394,11.120235,3191.307918,72.353343,161.343109,0.002933,0.005865,0.029326,0.035191,0.580645,0.017595,0.325513,0.002933,0.211144,0.020528,0.683284,0.085044,0.005865,0.055718,0.023460,0.0,0.014663,0.002933,0.005865,0.0,0.029326,0.011730,0.017595,0.049853,0.017595,0.008798,0.008798,0.011730,0.011730,...,0.005865,0.043988,0.011730,0.043988,0.005865,0.061584,0.002933,0.041056,0.008798,0.017595,0.011730,0.002933,0.011730,0.002933,0.129032,0.026393,0.0,0.023460,0.002933,0.002933,0.076246,0.061584,0.008798,0.011730,0.002933,0.002933,0.759531,0.152493,0.085044,0.906158,0.005865,0.005865,0.002933,0.020528,0.002933,0.055718,0.020528,0.020528,0.958944,4.072434
std,137.231746,5.953827,46.435453,0.742529,0.451598,88.161129,9.192305,61.320855,932.533527,9.357839,22.305556,0.054153,0.076471,0.168965,0.184532,0.494179,0.131668,0.469255,0.054153,0.408720,0.142006,0.465879,0.279357,0.076471,0.229714,0.151583,0.0,0.120375,0.054153,0.076471,0.0,0.168965,0.107827,0.131668,0.217962,0.131668,0.093520,0.093520,0.107827,0.107827,...,0.076471,0.205370,0.107827,0.205370,0.076471,0.240751,0.054153,0.198711,0.093520,0.131668,0.107827,0.054153,0.107827,0.054153,0.335728,0.160536,0.0,0.151583,0.054153,0.054153,0.265782,0.240751,0.093520,0.107827,0.054153,0.054153,0.427996,0.360026,0.279357,0.292037,0.076471,0.076471,0.054153,0.142006,0.054153,0.229714,0.142006,0.142006,0.198711,0.377066
min,0.000000,2.000000,0.000000,2.400000,1.000000,143.000000,115.600000,1.000000,1020.000000,23.600000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000

In [0]:
train_new['liked'] = np.where(train_new['Rating']>=4.0, 1, 0)

In [187]:
train_new['liked'].value_counts()

1    238
0    103
Name: liked, dtype: int64

In [188]:
train_new.shape,test_new.shape

((341, 89), (119, 87))

# Solution

In [0]:
class MPNeuron:
    
    def __init__(self):
        self.b = None

    def model(self,x):
        if np.sum(x)>=self.b:
          return 1
        else:
          return 0
  
    def predict(self,X):
        y_pred = []
        for x in X:
            result = self.model(x)
            y_pred.append(result)
        return np.array(y_pred)

    def fit(self,X,Y):
        accuracy = {}
        for b in range(X.shape[1]):
            self.b = b
            accuracy[b] = accuracy_score(self.predict(X),Y)
            print(b, accuracy[b])
        best_b = max(accuracy, key = accuracy.get)
        self.b = best_b

        print("Optimal b", best_b)
        print("Max accuracy", accuracy[best_b])

    def train_test_analyse(self,X_train,Y_train,X_test,Y_test,test_size=None):
        y_train_preds = []
        y_test_preds = []
        optimal_train_b = 0
        optimal_test_b = 0
        max_train_accuracy = 0
        max_test_accuracy = 0 
        train_accuracies = []
        test_accuracies = []
        for b in range(x_btrain.shape[1]):
            train_correct = 0
            test_correct = 0
        for x,y in zip(x_btrain,y_train):
            y_pred = (np.sum(x)>=b)
            y_train_preds.append(y_pred)
            if y_pred==y:
                train_correct+=1
                accuracy = train_correct/x_btrain.shape[0]
                train_accuracies.append(accuracy)
            if accuracy>max_train_accuracy:
                max_train_accuracy = accuracy
                optimal_train_b = b

        for x,y in zip(x_btest,y_test):
            y_pred = (np.sum(x)>=b)
            y_test_preds.append(y_pred)
            if y_pred==y:
                test_correct+=1
                accuracy = test_correct/x_btest.shape[0]
                test_accuracies.append(accuracy)
            if accuracy>max_test_accuracy:
                max_test_accuracy = accuracy
                optimal_test_b = b

        plt.plot(train_accuracies)
        plt.plot(test_accuracies)
        plt.ylim([0,1])
        if test_size!=None:
              plt.savefig("Accuracy analysis for test size :"+str(test_size)+".png")
        plt.show()


In [0]:
mp = MPNeuron()

In [0]:
x = train_new.drop(['PhoneId','liked','Rating'],axis=1)
y = train_new['liked']
test_x = test_new.drop(['PhoneId'],axis=1)

In [214]:
x.head()

,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,...,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G
0,20,64,6.26,1.8,403,157.9,4,4000,80.68,182,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
1,8,32,6.20,1.8,271,156.2,3,4230,80.85,168,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
2,25,32,6.30,2.1,409,157.0,3,3500,83.68,168,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
3,24,64,6.00,2.2,411,159.8,4,3300,74.78,169,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1
4,16,64,6.50,2.2,396,160.4,4,3750,84.23,175,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1


In [0]:
mean_df = train_new.groupby('liked').mean()

In [216]:
mean_df

,PhoneId,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,...,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,Rating
liked,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,299.485437,7.601942,30.330097,5.163689,1.568447,327.116505,146.980823,27.543689,2984.805825,69.088544,158.84466,0.000000,0.019417,0.038835,0.009709,0.436893,0.009709,0.485437,0.000000,0.330097,0.029126,0.582524,0.058252,0.019417,0.000000,0.009709,0.0,0.038835,0.000000,0.019417,0.0,0.038835,0.000000,0.048544,0.019417,0.00000,0.029126,0.009709,0.038835,0.009709,...,0.000000,0.116505,0.029126,0.048544,0.000000,0.077670,0.009709,0.000000,0.000000,0.009709,0.019417,0.009709,0.000000,0.009709,0.058252,0.048544,0.0,0.009709,0.000000,0.009709,0.019417,0.019417,0.000000,0.029126,0.009709,0.000000,0.902913,0.038835,0.058252,0.932039,0.019417,0.019417,0.000000,0.019417,0.009709,0.000000,0.009709,0.048544,0.941748,3.620388
1,204.184874,10.453782,53.310924,5.591555,1.889538,351.609244,151.229813,4.012605,3280.676471,73.766261,162.42437,0.004202,0.000000,0.025210,0.046218,0.642857,0.021008,0.256303,0.004202,0.159664,0.016807,0.726891,0.096639,0.000000,0.079832,0.029412,0.0,0.004202,0.004202,0.000000,0.0,0.025210,0.016807,0.004202,0.063025,0.02521,0.000000,0.008403,0.000000,0.012605,...,0.008403,0.012605,0.004202,0.042017,0.008403,0.054622,0.000000,0.058824,0.012605,0.021008,0.008403,0.000000,0.016807,0.000000,0.159664,0.016807,0.0,0.029412,0.004202,0.000000,0.100840,0.079832,0.012605,0.004202,0.000000,0.004202,0.697479,0.201681,0.096639,0.894958,0.000000,0.000000,0.004202,0.021008,0.000000,0.079832,0.025210,0.008403,0.966387,4.268067


In [217]:
mean_df['PhoneId']

liked
0    299.485437
1    204.184874
Name: PhoneId, dtype: float64

In [0]:
def bin(train,out,feature=None,labels=None):
    
    cols = list(out.columns)
    if labels==None:
      labels=[]
      mean_df = train.groupby(feature).mean()
      
      for col in cols:
          if mean_df[col][0]>mean_df[col][1]:
              out[col] = train[col].map(lambda x:1 if x<np.mean(np.array(train[col])) else 0)
              labels.append(0)
          else:
              out[col] = train[col].map(lambda x:1 if x>np.mean(np.array(train[col])) else 0)
              labels.append(1)
      return labels,out

    else:
      for index,col in enumerate(cols):
        if labels[index]==0:
          out[col] = train[col].map(lambda x:1 if x<np.mean(np.array(train[col])) else 0)
        else:
          out[col] = train[col].map(lambda x:1 if x>np.mean(np.array(train[col])) else 1) 
      return out         

In [0]:
labels,x = bin(train_new,x,feature='liked')

In [220]:
x

,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,...,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G
0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1
1,0,0,1,1,0,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1
2,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1
3,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,1,1
4,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1
5,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,1,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1
6,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1
7,0,0,1,1,1,0,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,1,1
8,0,0,1,1,0,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,1,1
9,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,1,0,1,...,1,0,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1


In [221]:
x['Num_cores_Quad'].value_counts()

1    230
0    111
Name: Num_cores_Quad, dtype: int64

In [222]:
x['os_name_iOS'].value_counts()

0    322
1     19
Name: os_name_iOS, dtype: int64

In [0]:
x_train = x.values
y_train = y.values

In [224]:
mp.fit(x_train,y_train)

0 0.6979472140762464
1 0.6979472140762464
2 0.6979472140762464
3 0.6979472140762464
4 0.6979472140762464
5 0.6979472140762464
6 0.6979472140762464
7 0.6979472140762464
8 0.6979472140762464
9 0.6979472140762464
10 0.6979472140762464
11 0.6979472140762464
12 0.6979472140762464
13 0.6979472140762464
14 0.6979472140762464
15 0.6979472140762464
16 0.6979472140762464
17 0.6979472140762464
18 0.6979472140762464
19 0.6979472140762464
20 0.6979472140762464
21 0.6979472140762464
22 0.6979472140762464
23 0.6979472140762464
24 0.6979472140762464
25 0.6979472140762464
26 0.6979472140762464
27 0.6979472140762464
28 0.6979472140762464
29 0.6979472140762464
30 0.6979472140762464
31 0.6979472140762464
32 0.6979472140762464
33 0.6979472140762464
34 0.7038123167155426
35 0.7126099706744868
36 0.7126099706744868
37 0.7155425219941349
38 0.7243401759530792
39 0.7390029325513197
40 0.7155425219941349
41 0.7155425219941349
42 0.7126099706744868
43 0.7038123167155426
44 0.6803519061583577
45 0.624633431085044

In [0]:
x_test = bin(test_x,test_x,labels=labels)

In [226]:
x_test.head()

,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,...,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1


In [0]:
x_test_val = x_test.values

In [0]:
y_pred = mp.predict(x_test_val)

In [229]:
test_x

,Resolution,Internal Memory,Screen Size,Processor_frequency,Pixel Density,Height,RAM,Capacity,Screen to Body Ratio (calculated),Weight,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,...,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,Brand_OPPO,Brand_OnePlus,Brand_Oppo,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1


In [230]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1])

In [231]:
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':y_pred})
submission = submission[['PhoneId', 'Class']]
submission.head()

,PhoneId,Class
0,3,1
1,11,1
2,13,1
3,16,1
4,19,1


In [0]:
submission.to_csv("submission.csv", index=False)

In [233]:
y_pred.shape

(119,)